In [15]:
import torch

Basic Manipulation

In [16]:
lst = list(range(10))
print(f"list:{lst}")

a= torch.tensor(lst)
print(f"torch tensor:{a}")

list:[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
torch tensor:tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


Creating a 2D tensor (matrix)

In [17]:
tensor_2d = torch.tensor([[1,2,3],
                          [4,5,6]])
print(tensor_2d)
print(f"dimensions: {tensor_2d.ndimension()}")
print(f"shape: {tensor_2d.shape}")
print(f"size: {tensor_2d.size()}")

tensor([[1, 2, 3],
        [4, 5, 6]])
dimensions: 2
shape: torch.Size([2, 3])
size: torch.Size([2, 3])


Creating a 3D tensor

In [18]:
tensor_3d = torch.tensor([[[1,2,3],
                          [4,5,6]],
                          [[6,7,8],
                           [9,10,11]],
                          [[4,5,6],
                           [7,8,9]]])
print(tensor_3d)
print(f"dimensions: {tensor_3d.ndimension()}")
print(f"shape: {tensor_3d.shape}")
print(f"size: {tensor_3d.size()}")

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[ 4,  5,  6],
         [ 7,  8,  9]]])
dimensions: 3
shape: torch.Size([3, 2, 3])
size: torch.Size([3, 2, 3])


In [19]:
torch_4d = torch.tensor([[[[1,2,3],
                           [4,5,6]],
                           [[1,2,3],
                            [4,5,6]],
                          [[1,2,3],
                           [4,5,6]]]])
print(torch_4d)
print(f"dimensions: {torch_4d.ndimension()}")
print(f"shape: {torch_4d.shape}")
print(f"size: {torch_4d.size()}")

tensor([[[[1, 2, 3],
          [4, 5, 6]],

         [[1, 2, 3],
          [4, 5, 6]],

         [[1, 2, 3],
          [4, 5, 6]]]])
dimensions: 4
shape: torch.Size([1, 3, 2, 3])
size: torch.Size([1, 3, 2, 3])


In [20]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [21]:
training_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [22]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X[N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


Creating Models

In [23]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#define model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x= self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Optimizing model Parameters

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [25]:
def train(dataloader, model, loss_fn, optimizer):
  size=len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y= X.to(device), y.to(device)

    # compute prediction error
    pred = model(X)
    loss= loss_fn(pred, y)

    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch %100 ==0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


Test

In [28]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches= len(dataloader)
  model.eval()
  test_loss, correct= 0,0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      pred= model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [29]:
epochs= 5
for t in range(epochs):
  print(f"Epoch {t+1}\n...................")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done")

Epoch 1
...................
loss: 2.147064 [   64/60000]
loss: 2.131289 [ 6464/60000]
loss: 2.068712 [12864/60000]
loss: 2.094757 [19264/60000]
loss: 2.029883 [25664/60000]
loss: 1.970895 [32064/60000]
loss: 2.000089 [38464/60000]
loss: 1.909558 [44864/60000]
loss: 1.911184 [51264/60000]
loss: 1.841569 [57664/60000]
Test Error: 
 Accuracy: 58.0%, Avg loss: 1.841219

Epoch 2
...................
loss: 1.874852 [   64/60000]
loss: 1.837130 [ 6464/60000]
loss: 1.720132 [12864/60000]
loss: 1.767524 [19264/60000]
loss: 1.651612 [25664/60000]
loss: 1.615904 [32064/60000]
loss: 1.628354 [38464/60000]
loss: 1.535762 [44864/60000]
loss: 1.555570 [51264/60000]
loss: 1.454132 [57664/60000]
Test Error: 
 Accuracy: 61.3%, Avg loss: 1.481673

Epoch 3
...................
loss: 1.552251 [   64/60000]
loss: 1.515442 [ 6464/60000]
loss: 1.370027 [12864/60000]
loss: 1.445609 [19264/60000]
loss: 1.323597 [25664/60000]
loss: 1.333699 [32064/60000]
loss: 1.336466 [38464/60000]
loss: 1.270385 [44864/60000]
lo

In [31]:
torch.save(model.state_dict(), "model.pth")
print("save Pytorch Model state to model.pth")

save Pytorch Model state to model.pth


In [32]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [33]:
classes = [
    "T=shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f"Predicted: '{predicted}', Acutal: '{actual}'")


Predicted: 'Ankle boot', Acutal: 'Ankle boot'
